<a href="https://colab.research.google.com/github/chuuuuu/machine_learning_2021/blob/main/homework/hw01/hw1_emsembler_stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 1: COVID-19 Cases Prediction (Regression)**

In [ ]:
from os import path, makedirs
from google.colab import drive
drive.mount('/content/drive')
WORKSPACE = 'drive/MyDrive/ColabNotebooks/HW1'

if not path.exists(f'{WORKSPACE}/dataset'):
  makedirs(f'{WORKSPACE}/dataset')
  !gdown --id '19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF' --output '{WORKSPACE}/dataset/covid.train.csv'
  !gdown --id '1CE240jLm2npU-tdz81-oVKEF3T2yfT1O' --output '{WORKSPACE}/dataset/covid.test.csv'

DATA_PATH = f'{WORKSPACE}/dataset'

Downloading...
From: https://drive.google.com/uc?id=19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF
To: /content/covid.train.csv
100% 2.00M/2.00M [00:00<00:00, 7.58MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CE240jLm2npU-tdz81-oVKEF3T2yfT1O
To: /content/covid.test.csv
100% 651k/651k [00:00<00:00, 93.6MB/s]


In [ ]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import csv
import os

# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# set a random seed for reproducibility
myseed = 42069
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

os.makedirs('models', exist_ok=True)

config = {
    'INPUT_DIM': 14,
    'TRAIN_PATH': f'{DATA_PATH}/covid.train.csv',
    'TEST_PATH': f'{DATA_PATH}/covid.test.csv',
    'MODEL_PATH': 'models/model.pth',
    'PRED_PATH': 'pred.csv',

    'EPOCH_NUM': 30000,
    'BATCH_SIZE': 4096,
    'VAL_RATIO': 0.1,
    'OPTIMIZER': 'Adam',
    'OPTIM_PARAMS': {
        'lr': 5e-2,
        # 'weight_decay': 5e-3,
    },
    'DECAY_RATE': 0.999,
    'MIN_LR': 1e-4,
    'EARLY_STOP': 300,
    'MODEL_NUM': 3,
}

In [ ]:
class Drawer():
    def plot_learning_curve(self, loss_record, title=''):
        ''' Plot learning curve of your DNN (train & dev loss) '''
        total_steps = len(loss_record['train'])
        x_1 = range(total_steps)
        x_2 = x_1[::len(loss_record['train']) // len(loss_record['val'])]
        figure(figsize=(6, 4))
        plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
        plt.plot(x_2, loss_record['val'], c='tab:cyan', label='val')
        plt.ylim(0.0, 5.)
        plt.xlabel('Training steps')
        plt.ylabel('MSE loss')
        plt.title('Learning curve of {}'.format(title))
        plt.legend()
        plt.show()


    def plot_pred(self, dv_set, model, device, lim=35., preds=None, targets=None):
        ''' Plot prediction of your DNN '''
        if preds is None or targets is None:
            model.eval()
            preds, targets = [], []
            for x, y in dv_set:
                x, y = x.to(device), y.to(device)
                with torch.no_grad():
                    pred = model(x)
                    preds.append(pred.detach().cpu())
                    targets.append(y.detach().cpu())
            preds = torch.cat(preds, dim=0).numpy()
            targets = torch.cat(targets, dim=0).numpy()

        figure(figsize=(5, 5))
        plt.scatter(targets, preds, c='r', alpha=0.5)
        plt.plot([-0.2, lim], [-0.2, lim], c='b')
        plt.xlim(-0.2, lim)
        plt.ylim(-0.2, lim)
        plt.xlabel('ground truth value')
        plt.ylabel('predicted value')
        plt.title('Ground Truth v.s. Prediction')
        plt.show()

In [ ]:
from sklearn.feature_selection import f_regression, SelectKBest, mutual_info_regression
from sklearn.model_selection import train_test_split


class DataManager():
    def __init__(self):
        print('init data manager...')
        TRAIN_PATH = config['TRAIN_PATH']
        INPUT_DIM = config['INPUT_DIM']

        self.state = 1

        with open(TRAIN_PATH, 'r') as f:
            self.data = list(csv.reader(f))
            self.data = np.array(self.data[1:])[:, 1:].astype(np.float32)

        self.X = self.data[:, :-1]
        self.y = self.data[:, -1]

        selector = SelectKBest(f_regression, k=INPUT_DIM)
        selector.fit(self.X, self.y)
        self.cols = selector.get_support(indices=True)

        self.X = self.X[:, self.cols]

    def get_train_data(self):
        print('getting train data...')
        VAL_RATIO = config['VAL_RATIO']

        X_train, X_val, y_train, y_val = train_test_split(self.X, self.y, test_size=VAL_RATIO, random_state=self.state)
        self.state += 1

        return X_train, X_val, y_train, y_val
    
    def get_test_data(self):
        print('getting test data...')
        TEST_PATH = config['TEST_PATH']
        with open(TEST_PATH, 'r') as f:
            data = list(csv.reader(f))
            data = np.array(data[1:])[:, 1:].astype(np.float32)
        X_test = data[:, self.cols]
        return X_test


In [ ]:
class CovidDataset(Dataset):
    def __init__(self, X, y=None):
        print('init dataset...')
        self.X = torch.from_numpy(X).float()
        if y is None:
            self.y = None
        else:
            self.y = torch.from_numpy(y).float()

    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]

        return self.X[idx], self.y[idx]

    def __len__(self):
        return len(self.X)

In [ ]:
from torchsummary import summary

class NeuralNet(nn.Module):
    def __init__(self):
        print('init neural net...')
        super(NeuralNet, self).__init__()

        INPUT_DIM = config['INPUT_DIM']

        self.net = nn.Sequential(
            nn.BatchNorm1d(INPUT_DIM),
            nn.Linear(INPUT_DIM, 64),
            nn.ReLU(),
            # nn.BatchNorm1d(64),
            # nn.Linear(128, 128),
            # nn.ReLU(),
            # nn.Linear(32, 16),
            # nn.ReLU(),
            nn.Linear(64, 1),
        )

        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        return self.net(x).squeeze(1)

    def get_loss(self, y_pred, y):
        return self.criterion(y_pred, y)

    def summary(self):
        INPUT_DIM = config['INPUT_DIM']
        summary(self, (INPUT_DIM, ))

In [ ]:
class Trainer():
    def __init__(self):
        print('init trainer')
        self.set_device()
        self.set_model()
        self.set_data_loader()
        self.set_drawer()
        self.set_optim()

        self.loss_record = {'train': [], 'val': []}

    def set_drawer(self):
        self.drawer = Drawer()

    def draw_learning_curve(self):
        self.drawer.plot_learning_curve(self.loss_record, title='deep model')

    def draw_val_results(self):
        MODEL_PATH = config['MODEL_PATH']
        del self.model
        self.set_model()

        ckpt = torch.load(MODEL_PATH, map_location='cpu')
        self.model.load_state_dict(ckpt)
        self.drawer.plot_pred(self.val_loader, self.model, self.device)

    def pred_y_test(self):
        print('predicting...')
        BATCH_SIZE = config['BATCH_SIZE']
        PRED_PATH = config['PRED_PATH']

        X_test = self.dataManager.get_test_data()
        test_loader = DataLoader(X_test, BATCH_SIZE, False, drop_last=False, num_workers=0, pin_memory=True)

        self.model.eval()
        y_preds = []
        for x in test_loader:
            x = x.to(self.device)
            with torch.no_grad():
                y_pred = self.model(x)
                y_preds.append(y_pred.detach().cpu())
        
        y_preds = torch.cat(y_preds, dim=0).numpy()
        return y_preds

    def set_device(self):
        ''' Get device (if GPU is available, use GPU) '''
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def set_data_loader(self):
        BATCH_SIZE = config['BATCH_SIZE']
        self.dataManager = DataManager()
        X_train, X_val, y_train, y_val = self.dataManager.get_train_data()   

        train_set = CovidDataset(X_train, y_train)
        self.train_loader = DataLoader(train_set, BATCH_SIZE, True, drop_last=False, num_workers=0, pin_memory=True)

        val_set = CovidDataset(X_val, y_val)
        self.val_loader = DataLoader(val_set, BATCH_SIZE, False, drop_last=False, num_workers=0, pin_memory=True)

    def set_model(self):
        self.model = NeuralNet().to(self.device)

    def set_optim(self):
        OPTIMIZER = config['OPTIMIZER']
        OPTIM_PARAMS = config['OPTIM_PARAMS']

        self.optimizer = getattr(torch.optim, OPTIMIZER)(self.model.parameters(), **OPTIM_PARAMS)

    def train(self):
        EPOCH_NUM = config['EPOCH_NUM']
        MODEL_PATH = config['MODEL_PATH']
        EARLY_STOP = config['EARLY_STOP']

        min_val_loss = float('inf')
        early_stop_count = 0
        for epoch in range(EPOCH_NUM):
            self.model.train()
            for x, y in self.train_loader:
                self.optimizer.zero_grad()
                x, y = x.to(self.device), y.to(self.device)
                y_pred = self.model(x)
                loss = self.model.get_loss(y_pred, y)
                loss.backward()
                self.optimizer.step()

            self.update_lr()
            val_loss = self.get_loss(self.val_loader)
            train_loss = self.get_loss(self.train_loader)
            if epoch % 100 == 0:
                print(f'epoch: {epoch+1}, train_loss: {train_loss}, val_loss: {val_loss}')

            if val_loss < min_val_loss:
                min_val_loss = val_loss
                print(f'Saving model, epoch: {epoch+1}, train_loss: {train_loss}, val_loss: {val_loss}')
                torch.save(self.model.state_dict(), MODEL_PATH)
                early_stop_cnt = 0

            else:
                early_stop_cnt += 1

            self.loss_record['val'].append(val_loss)
            self.loss_record['train'].append(train_loss)

            if early_stop_cnt > EARLY_STOP:
                break

        # print(f'Saving model, epoch: {epoch+1}, train_loss: {train_loss}, val_loss: {val_loss}')
        # torch.save(self.model.state_dict(), MODEL_PATH)
        
        print(f'finished training after {epoch+1} epochs')
        self.model.summary()

    
    def update_lr(self):
        DECAY_RATE = config['DECAY_RATE']
        MIN_LR = config['MIN_LR']
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * DECAY_RATE
            param_group['lr'] = max(MIN_LR, param_group['lr'])

    def get_loss(self, loader):
        self.model.eval()
        total_loss = 0
        for x, y in loader:
            x, y = x.to(self.device), y.to(self.device)
            with torch.no_grad():
                y_pred = self.model(x)
                loss = self.model.get_loss(y_pred, y)
            
            total_loss += loss.detach().cpu().item() * len(x)
        total_loss /= len(loader.dataset)

        return total_loss

    def get_y_pred(self, loader):
        self.model.eval()
        y_preds = []
        for x, y in loader:
            x, y = x.to(self.device), y.to(self.device)
            with torch.no_grad():
                y_pred = self.model(x)
                y_preds.append(y_pred.detach().cpu())

        y_preds = torch.cat(y_preds, dim=0).numpy()
        return y_preds


In [ ]:
class Emssembler(Trainer):
    def __init__(self):
        MODEL_NUM = config['MODEL_NUM']
        self.trainers = []
        for i in range(MODEL_NUM):
            self.trainers.append(Trainer())
        
        self.trainModels()
        super(Emssembler, self).__init__()

    def set_model(self):
        self.model = MetaLearner().to(self.device)

    def set_data_loader(self):
        BATCH_SIZE = config['BATCH_SIZE']
        MODEL_NUM = config['MODEL_NUM']
        self.dataManager = DataManager()
        X_train, X_val, y_train, y_val = self.dataManager.get_train_data()   
        train_set = CovidDataset(X_train, y_train)
        train_loader = DataLoader(train_set, BATCH_SIZE, False, drop_last=False, num_workers=0, pin_memory=True)
        val_set = CovidDataset(X_val, y_val)
        val_loader = DataLoader(val_set, BATCH_SIZE, False, drop_last=False, num_workers=0, pin_memory=True)

        meta_X_train = None
        meta_X_val = None
        for i in range(MODEL_NUM):
            train_y_pred = self.trainers[i].get_y_pred(train_loader)
            train_y_pred = np.reshape(train_y_pred, (train_y_pred.shape[0], 1))
            if meta_X_train is None:
                meta_X_train = train_y_pred
            else:
                meta_X_train = np.concatenate((meta_X_train, train_y_pred), axis=1)

            val_y_pred = self.trainers[i].get_y_pred(val_loader)
            val_y_pred = np.reshape(val_y_pred, (val_y_pred.shape[0], 1))
            if meta_X_val is None:
                meta_X_val = val_y_pred
            else:
                meta_X_val = np.concatenate((meta_X_val, val_y_pred), axis=1)

        meta_train_set = CovidDataset(meta_X_train, y_train)
        meta_val_set = CovidDataset(meta_X_val, y_val)
        self.train_loader = DataLoader(meta_train_set, BATCH_SIZE, True, drop_last=False, num_workers=0, pin_memory=True)
        self.val_loader = DataLoader(meta_val_set, BATCH_SIZE, False, drop_last=False, num_workers=0, pin_memory=True)
        
    def trainModels(self):
        MODEL_NUM = config['MODEL_NUM']
        for trainer in self.trainers:
            trainer.train()

    def pred_y_test(self):
        MODEL_NUM = config['MODEL_NUM']
        PRED_PATH = config['PRED_PATH']
        BATCH_SIZE = config['BATCH_SIZE']
        
        # y_preds = None
        meta_X_test = None
        for trainer in self.trainers:
            test_y_pred = trainer.pred_y_test()
            test_y_pred = np.reshape(test_y_pred, (test_y_pred.shape[0], 1))
            if meta_X_test is None:
                meta_X_test = test_y_pred
            else:
                meta_X_test = np.concatenate((meta_X_test, test_y_pred), axis=1)

        test_loader = DataLoader(meta_X_test, BATCH_SIZE, False, drop_last=False, num_workers=0, pin_memory=True)

        self.model.eval()
        meta_y_preds = []
        for x in test_loader:
            x = x.to(self.device)
            with torch.no_grad():
                meta_y_pred = self.model(x)
                meta_y_preds.append(meta_y_pred.detach().cpu())
        
        meta_y_preds = torch.cat(meta_y_preds, dim=0).numpy()

        with open(PRED_PATH, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(['id', 'tested_positive'])
            for i, p in enumerate(meta_y_preds):
                writer.writerow([i, p])


In [ ]:
from torchsummary import summary

class MetaLearner(nn.Module):
    def __init__(self):
        print('init neural net...')
        super(MetaLearner, self).__init__()

        MODEL_NUM = config['MODEL_NUM']

        self.net = nn.Sequential(
            nn.Linear(MODEL_NUM, 1),
            # nn.ReLU(),
            # nn.Linear(16, 1),
        )

        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        return self.net(x).squeeze(1)

    def get_loss(self, y_pred, y):
        return self.criterion(y_pred, y)

    def summary(self):
        MODEL_NUM = config['MODEL_NUM']
        summary(self, (MODEL_NUM, ))


In [ ]:
# trainer = Trainer()
# trainer.train()
# trainer.draw_learning_curve()
# trainer.draw_val_results()
# trainer.pred_y_test()
emssembler = Emssembler()
emssembler.train()
emssembler.pred_y_test()
print(f'config: {config}')



Streaming output truncated to the last 5000 lines.
Saving model, epoch: 16731, train_loss: 0.9022015929222107, val_loss: 0.8746574521064758
Saving model, epoch: 16732, train_loss: 0.9022002816200256, val_loss: 0.8746563792228699
Saving model, epoch: 16733, train_loss: 0.9021990895271301, val_loss: 0.8746554255485535
Saving model, epoch: 16734, train_loss: 0.9021977782249451, val_loss: 0.8746544122695923
Saving model, epoch: 16735, train_loss: 0.9021965861320496, val_loss: 0.8746535778045654
Saving model, epoch: 16736, train_loss: 0.9021952748298645, val_loss: 0.8746525645256042
Saving model, epoch: 16737, train_loss: 0.9021940231323242, val_loss: 0.8746516108512878
Saving model, epoch: 16738, train_loss: 0.9021927714347839, val_loss: 0.8746504783630371
Saving model, epoch: 16739, train_loss: 0.9021915197372437, val_loss: 0.874649703502655
Saving model, epoch: 16740, train_loss: 0.9021902084350586, val_loss: 0.8746485710144043
Saving model, epoch: 16741, train_loss: 0.9021891355514526, 

In [ ]:
model = emssembler.model
for param in model.parameters():
  print(param.data)

tensor([[0.6077, 0.0980, 0.2946]], device='cuda:0')
tensor([-0.0056], device='cuda:0')
